# Step 1: Imports

In [1]:
from typing import List, Tuple
import numpy as np
import pandas as pd


COMPETITION_NAME = "warmup-round-march-machine-learning-mania-2023"

pd.set_option("display.max_columns", None)


def load_raw_data(filename: str, competition_name:str = COMPETITION_NAME) -> pd.DataFrame:
    mens_filepath = f"/kaggle/input/{competition_name}/M{filename}.csv"
    womens_filepath = f"/kaggle/input/{competition_name}/W{filename}.csv"
    df_mens = pd.read_csv(mens_filepath)
    df_womens = pd.read_csv(womens_filepath)
    return pd.concat([df_mens, df_womens])


def process_detailed_results(detailed_results: pd.DataFrame) -> pd.DataFrame:
    df = detailed_results.copy()
    df = clean_detailed_results(df)
    df = game_to_team_conversion(df)
    df = transform_team_results(df)
    return df


def clean_detailed_results(df: pd.DataFrame) -> pd.DataFrame:
    return df.drop(["WLoc", "DayNum"], axis=1)


def game_to_team_conversion(game_results: pd.DataFrame) -> pd.DataFrame:
    winners = rename_columns(game_results, "W")
    loosers = rename_columns(game_results, "L")
    team_results = pd.concat((winners, loosers))
    team_results.drop(["TeamIDOpp"], axis=1, inplace=True)
    return team_results


def transform_team_results(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(["Season", "TeamID"]).median()
    df["FGP"] =  df["FGM"] / df["FGA"]
    df["FGP3"] =  df["FGM3"] / df["FGA3"]
    df["FTP"] =  df["FTM"] / df["FTA"]
    df["FGPOpp"] =  df["FGMOpp"] / df["FGAOpp"]
    df["FGP3Opp"] =  df["FGM3Opp"] / df["FGA3Opp"]
    df["FTPOpp"] =  df["FTMOpp"] / df["FTAOpp"]
    return df.reset_index()
    
    
def rename_columns(df: pd.DataFrame, team_prefix: str) -> pd.DataFrame:
    df = df.copy()
    df.columns =  (rename_column(column_name, team_prefix) for column_name in df.columns)
    return df


def rename_column(column_name: str, team_prefix: str) -> pd.DataFrame:
    if team_prefix == "W":
        opponent_prefix = "L"
    elif team_prefix == "L":
        opponent_prefix = "W"
    else:
        raise ValueError
    if column_name.startswith(team_prefix):
        column_name = column_name.lstrip(team_prefix)
    elif column_name.startswith(opponent_prefix):
        column_name = f"{column_name.lstrip(opponent_prefix)}Opp"
    return column_name


def split_winner_and_looser_columns(df: pd.DataFrame) -> Tuple[List[str], List[str]]:
    winner_columns = [name for name in df.columns if not name.startswith("L")]
    looser_columns = [name for name in df.columns if not name.startswith("W")]
    return winner_columns, looser_columns


def clean_column_names(df: pd.DataFrame) -> List[str]:
    column_names = [
        name[1:] if 
        name.startswith("L") or name.startswith("W")
        else name 
        for name in df.columns
    ]
    return column_names


def process_seeds(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    mask = df["Season"] > 2002
    df = df[mask]
    df["Seed"] = df["Seed"].str.replace(r"\D+","", regex=True)
    df["Seed"] = df["Seed"].astype(int)
    return df


def process_rankings(df_in:pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    mask = df["RankingDayNum"] == df["RankingDayNum"].max()
    df = df[mask]
    df.drop(["SystemName", "RankingDayNum"], axis=1, inplace=True)
    df = df.groupby(["Season", "TeamID"]).agg("median")
    return df.reset_index()


def merge_features(
    season_features: pd.DataFrame, 
    tournament_features: pd.DataFrame, 
    seed_features: pd.DataFrame, 
    ranking_features: pd.DataFrame
) -> pd.DataFrame:
    features = pd.merge(
        season_features,
        tournament_features,
        how="inner",
        on=["Season", "TeamID"],
        suffixes=("Reg", "Tou")
    )
    features = features.merge(
        seed_features,
        how="inner",
        on=["Season", "TeamID"]
    )
    features = features.merge(
        ranking_features,
        how="left",
        on=["Season", "TeamID"],
    )
    return features


def get_outcomes(df):
    input_rows = df.to_records()
    output_rows = [parse_row(input_row) for input_row in input_rows]
    out_df = pd.DataFrame(output_rows)
    return out_df


def parse_row(row):
    season = row['Season']
    winning_team_id = row['WTeamID']
    losing_team_id = row['LTeamID']
    if winning_team_id < losing_team_id:
        small_id = winning_team_id
        big_id = losing_team_id
        outcome = True
    elif losing_team_id < winning_team_id:
        small_id = losing_team_id
        big_id = winning_team_id
        outcome = False
    record = {
            "ID": f"{season}_{small_id}_{big_id}",
            'Season': season,
            'LowID': small_id,
            'HighID': big_id,
            'Win': outcome
    }
    return record


def merge_outcomes_with_features(outcomes: pd.DataFrame, features: pd.DataFrame, how: str = "inner") -> pd.DataFrame:
    feature_names = [name for name in features.columns if name not in ["Season", "TeamID", "Gender"]]
    data = pd.merge(
        outcomes, 
        features, 
        how=how, 
        left_on=["Season", "HighID"], 
        right_on=["Season", "TeamID"]
    )
    data = pd.merge(
        data, 
        features, 
        how=how, 
        left_on=["Season", "LowID"], 
        right_on=["Season", "TeamID"],
        suffixes=("High", "Low")
    )
    for name in feature_names:
        data[f"{name}Diff"] = data[f"{name}High"] - data[f"{name}Low"]
        data.drop([f"{name}High", f"{name}Low"], axis=1, inplace=True)
    data.drop(
        ["Season", "HighID", "LowID","TeamIDHigh","TeamIDLow"], 
        axis=1, 
        inplace=True
    )
    data.set_index("ID", inplace=True)
    return data

# Step 2: Load the data

## Season Detailed Results

In [2]:
RegularSeasonDetailedResults = load_raw_data("RegularSeasonDetailedResults")
RegularSeasonDetailedResults.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
70673,2023,127,3415,63,3142,54,N,0,20,56,2,10,21,30,11,23,9,14,5,2,22,17,43,4,13,16,21,1,21,5,15,7,2,24
70674,2023,127,3424,71,3361,68,H,0,23,55,2,12,23,33,10,23,12,14,7,6,16,28,58,5,10,7,10,7,24,12,13,4,1,26
70675,2023,127,3455,65,3378,53,A,0,24,51,6,13,11,13,2,28,14,11,7,2,12,19,60,4,22,11,13,8,22,11,12,2,6,12
70676,2023,127,3461,65,3161,56,H,0,25,57,5,17,10,16,13,35,15,14,3,2,13,21,55,9,24,5,8,1,20,12,8,8,4,17
70677,2023,127,3477,65,3230,62,A,0,23,50,3,13,16,19,12,20,10,9,6,0,11,22,51,8,22,10,12,8,15,12,9,5,1,13


In [3]:
season_features = process_detailed_results(RegularSeasonDetailedResults)
season_features.tail()

,Season,TeamID,Score,ScoreOpp,NumOT,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,FGMOpp,FGAOpp,FGM3Opp,FGA3Opp,FTMOpp,FTAOpp,OROpp,DROpp,AstOpp,TOOpp,StlOpp,BlkOpp,PFOpp,FGP,FGP3,FTP,FGPOpp,FGP3Opp,FTPOpp
12130,2023,3473,56.0,71.0,0.0,21.0,51.0,7.0,21.0,8.0,13.0,6.0,19.0,12.0,19.0,6.0,2.0,16.0,27.0,61.0,5.0,17.0,11.0,17.0,10.0,21.0,13.0,13.0,11.0,2.0,17.0,0.411765,0.333333,0.615385,0.442623,0.294118,0.647059
12131,2023,3474,57.5,73.0,0.0,20.0,58.5,4.0,15.0,12.5,17.5,8.0,20.0,7.0,14.0,7.0,2.0,21.5,24.5,55.5,5.0,14.5,16.0,24.0,9.5,28.0,14.0,17.0,6.0,3.0,18.5,0.341880,0.266667,0.714286,0.441441,0.344828,0.666667
12132,2023,3475,59.5,63.0,0.0,21.0,55.5,4.0,16.0,15.0,19.0,9.0,24.0,13.0,20.5,7.0,2.0,19.0,22.0,56.0,7.0,22.5,14.0,21.5,9.0,20.5,12.0,16.0,11.0,3.0,20.0,0.378378,0.250000,0.789474,0.392857,0.311111,0.651163
12133,2023,3476,61.0,66.0,0.0,22.0,56.5,6.0,21.0,9.5,12.0,9.0,21.0,13.5,14.5,3.0,3.0,14.0,26.0,57.5,5.0,14.0,10.5,15.0,8.0,22.0,12.0,11.0,7.0,3.0,15.5,0.389381,0.285714,0.791667,0.452174,0.357143,0.700000
12134,2023,3477,65.0,71.0,0.0,22.0,58.0,4.0,14.0,14.0,21.0,11.0,23.0,11.0,16.0,7.0,3.0,15.0,27.0,60.0,7.0,19.0,10.0,13.0,9.0,24.0,15.0,16.0,8.0,4.0,19.0,0.379310,0.285714,0.666667,0.450000,0.368421,0.769231


## Tournament Detailed Results

In [4]:
NCAATourneyDetailedResults = load_raw_data("NCAATourneyDetailedResults")
NCAATourneyDetailedResults.tail()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
755,2022,147,3163,91,3301,87,N,2,37,77,5,21,12,20,12,23,10,7,5,2,16,32,66,7,23,16,19,6,30,20,13,4,7,16
756,2022,147,3257,62,3276,50,N,0,25,58,5,15,7,9,6,20,12,11,15,4,17,16,46,3,14,15,20,10,24,9,21,6,2,12
757,2022,151,3163,63,3390,58,N,0,21,57,5,14,16,20,12,30,14,19,5,2,16,23,66,4,23,8,13,11,23,10,11,11,3,16
758,2022,151,3376,72,3257,59,N,0,27,57,6,17,12,17,8,24,19,14,11,4,11,27,63,1,8,4,7,11,18,5,15,13,2,17
759,2022,153,3376,64,3163,49,N,0,22,60,3,16,17,26,18,23,9,14,6,4,11,22,54,4,16,1,4,3,16,14,14,4,5,21


In [5]:
tournament_features = process_detailed_results(NCAATourneyDetailedResults)
tournament_features["Season"] += 1
tournament_features.tail()

,Season,TeamID,Score,ScoreOpp,NumOT,FGM,FGA,FGM3,FGA3,FTM,FTA,OR,DR,Ast,TO,Stl,Blk,PF,FGMOpp,FGAOpp,FGM3Opp,FGA3Opp,FTMOpp,FTAOpp,OROpp,DROpp,AstOpp,TOOpp,StlOpp,BlkOpp,PFOpp,FGP,FGP3,FTP,FGPOpp,FGP3Opp,FTPOpp
2034,2023,3426,71.0,78.0,0.0,27.0,62.0,7.0,16.0,10.0,16.0,7.0,26.0,11.0,9.0,6.0,5.0,20.0,28.0,65.0,7.0,20.0,15.0,26.0,11.0,27.0,6.0,7.0,2.0,6.0,14.0,0.435484,0.437500,0.625000,0.430769,0.350000,0.576923
2035,2023,3428,74.0,73.5,0.0,27.5,53.5,10.5,25.5,8.5,12.0,6.0,25.5,16.5,17.5,3.0,1.5,14.5,31.5,67.0,3.5,14.5,7.0,12.5,9.5,18.5,10.5,8.5,9.0,4.5,12.0,0.514019,0.411765,0.708333,0.470149,0.241379,0.560000
2036,2023,3437,55.0,60.5,0.0,20.5,58.5,6.0,23.5,8.0,12.0,7.5,19.5,11.0,10.0,8.5,5.0,18.0,22.5,55.5,4.0,15.0,11.5,16.5,11.5,29.5,12.5,15.0,7.0,1.0,12.5,0.350427,0.255319,0.666667,0.405405,0.266667,0.696970
2037,2023,3439,81.0,84.0,0.0,30.0,60.0,6.0,17.0,15.0,20.0,7.0,28.0,13.0,10.0,1.0,1.0,13.0,30.0,64.0,15.0,38.0,9.0,11.0,2.0,25.0,19.0,3.0,7.0,1.0,18.0,0.500000,0.352941,0.750000,0.468750,0.394737,0.818182
2038,2023,3450,40.0,50.0,0.0,14.0,56.0,3.0,22.0,9.0,14.0,8.0,31.0,5.0,17.0,2.0,2.0,23.0,14.0,53.0,3.0,14.0,19.0,22.0,6.0,30.0,7.0,11.0,6.0,3.0,11.0,0.250000,0.136364,0.642857,0.264151,0.214286,0.863636


## Tournement Seeds

In [6]:
NCAATourneySeeds = load_raw_data("NCAATourneySeeds")
NCAATourneySeeds.tail()

,Season,Seed,TeamID
1535,2022,Z12,3125
1536,2022,Z13,3138
1537,2022,Z14,3110
1538,2022,Z15,3218
1539,2022,Z16,3107


In [7]:
seed_features = process_seeds(NCAATourneySeeds)
seed_features.tail()

,Season,Seed,TeamID
1535,2022,12,3125
1536,2022,13,3138
1537,2022,14,3110
1538,2022,15,3218
1539,2022,16,3107


## Team Rankings

In [8]:
MMasseyOrdinals = pd.read_csv("/kaggle/input/warmup-round-march-machine-learning-mania-2023/MMasseyOrdinals.csv")
MMasseyOrdinals.tail()

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
4922144,2023,128,WOL,1473,332
4922145,2023,128,WOL,1474,166
4922146,2023,128,WOL,1475,260
4922147,2023,128,WOL,1476,301
4922148,2023,128,WOL,1477,303


In [9]:
ranking_features = process_rankings(MMasseyOrdinals)
ranking_features.tail()

,Season,TeamID,OrdinalRank
6533,2022,1468,188.0
6534,2022,1469,272.0
6535,2022,1470,225.0
6536,2022,1471,263.0
6537,2022,1472,312.5


## Merge features

In [10]:
features = merge_features(season_features, tournament_features, seed_features, ranking_features)
features.head()

,Season,TeamID,ScoreReg,ScoreOppReg,NumOTReg,FGMReg,FGAReg,FGM3Reg,FGA3Reg,FTMReg,FTAReg,ORReg,DRReg,AstReg,TOReg,StlReg,BlkReg,PFReg,FGMOppReg,FGAOppReg,FGM3OppReg,FGA3OppReg,FTMOppReg,FTAOppReg,OROppReg,DROppReg,AstOppReg,TOOppReg,StlOppReg,BlkOppReg,PFOppReg,FGPReg,FGP3Reg,FTPReg,FGPOppReg,FGP3OppReg,FTPOppReg,ScoreTou,ScoreOppTou,NumOTTou,FGMTou,FGATou,FGM3Tou,FGA3Tou,FTMTou,FTATou,ORTou,DRTou,AstTou,TOTou,StlTou,BlkTou,PFTou,FGMOppTou,FGAOppTou,FGM3OppTou,FGA3OppTou,FTMOppTou,FTAOppTou,OROppTou,DROppTou,AstOppTou,TOOppTou,StlOppTou,BlkOppTou,PFOppTou,FGPTou,FGP3Tou,FTPTou,FGPOppTou,FGP3OppTou,FTPOppTou,Seed,OrdinalRank
0,2004,1104,72.0,70.0,0.0,25.0,55.0,7.0,17.0,15.0,21.0,12.0,24.0,11.0,12.0,6.0,3.0,19.0,24.0,57.0,7.0,20.0,15.0,19.0,11.0,22.0,11.0,14.0,5.0,3.0,21.0,0.454545,0.411765,0.714286,0.421053,0.350000,0.789474,62.0,67.0,0.0,22.0,52.0,5.0,12.0,13.0,16.0,9.0,20.0,13.0,8.0,2.0,6.0,21.0,19.0,49.0,7.0,18.0,22.0,26.0,13.0,22.0,15.0,8.0,1.0,2.0,17.0,0.423077,0.416667,0.812500,0.387755,0.388889,0.846154,8,33.0
1,2004,1112,85.0,78.0,0.0,31.0,64.0,7.0,20.0,16.0,21.0,14.0,25.0,18.0,15.0,7.0,4.0,17.0,29.0,65.0,8.0,24.0,12.0,18.0,14.0,22.0,18.0,16.0,7.0,2.0,20.0,0.484375,0.350000,0.761905,0.446154,0.333333,0.666667,84.0,74.5,0.0,32.5,70.0,7.0,22.5,15.0,18.5,12.5,29.5,18.5,13.0,9.0,4.5,17.0,27.0,65.5,9.0,24.5,11.5,18.0,12.0,26.5,15.5,16.0,8.0,4.5,18.0,0.464286,0.311111,0.810811,0.412214,0.367347,0.638889,9,35.0
2,2004,1140,74.0,65.0,0.0,25.0,52.0,5.0,15.0,16.0,24.0,10.0,23.0,15.0,13.0,7.0,3.0,19.0,23.0,51.0,6.0,16.0,12.0,17.0,9.0,20.0,12.0,14.0,7.0,2.0,20.0,0.480769,0.333333,0.666667,0.450980,0.375000,0.705882,53.0,58.0,0.0,20.0,64.0,2.0,17.0,11.0,13.0,15.0,26.0,11.0,11.0,8.0,4.0,22.0,17.0,52.0,4.0,14.0,20.0,27.0,12.0,29.0,8.0,14.0,3.0,8.0,16.0,0.312500,0.117647,0.846154,0.326923,0.285714,0.740741,12,34.0
3,2004,1153,79.5,61.5,0.0,27.0,59.5,8.0,19.0,15.0,23.0,12.0,24.5,15.5,12.0,8.5,5.0,18.5,19.5,52.5,6.0,18.0,15.0,22.5,11.0,23.0,10.0,17.0,5.0,2.0,20.5,0.453782,0.421053,0.652174,0.371429,0.333333,0.666667,69.0,74.0,0.0,26.0,66.0,10.0,27.0,7.0,10.0,13.0,22.0,13.0,10.0,7.0,6.0,24.0,20.0,47.0,6.0,14.0,28.0,37.0,8.0,28.0,12.0,12.0,2.0,2.0,15.0,0.393939,0.370370,0.700000,0.425532,0.428571,0.756757,4,12.0
4,2004,1163,76.0,63.0,0.0,29.0,62.0,6.0,15.0,13.0,21.0,16.0,30.0,18.0,15.0,6.0,9.0,16.0,24.0,65.0,5.0,19.0,10.0,15.0,14.0,22.0,12.0,12.0,7.0,4.0,18.0,0.467742,0.400000,0.619048,0.369231,0.263158,0.666667,78.0,74.0,0.0,30.0,68.0,4.0,13.0,17.0,25.0,14.0,27.0,12.0,11.0,4.0,8.0,16.0,24.0,64.0,5.0,18.0,12.0,17.0,15.0,26.0,12.0,11.0,5.0,5.0,21.0,0.441176,0.307692,0.680000,0.375000,0.277778,0.705882,2,8.0


## Build Dataset

In [11]:
from sklearn.model_selection import train_test_split

data = load_raw_data("NCAATourneyCompactResults")
data_train, data_valid = train_test_split(data, random_state=0)

outcomes_train = get_outcomes(data_train)
outcomes_valid = get_outcomes(data_valid)

In [12]:
features_train = merge_outcomes_with_features(outcomes_train, features)
features_valid = merge_outcomes_with_features(outcomes_valid, features)
print(features_train.shape)
features_train.tail()

(550, 73)


,Win,ScoreRegDiff,ScoreOppRegDiff,NumOTRegDiff,FGMRegDiff,FGARegDiff,FGM3RegDiff,FGA3RegDiff,FTMRegDiff,FTARegDiff,ORRegDiff,DRRegDiff,AstRegDiff,TORegDiff,StlRegDiff,BlkRegDiff,PFRegDiff,FGMOppRegDiff,FGAOppRegDiff,FGM3OppRegDiff,FGA3OppRegDiff,FTMOppRegDiff,FTAOppRegDiff,OROppRegDiff,DROppRegDiff,AstOppRegDiff,TOOppRegDiff,StlOppRegDiff,BlkOppRegDiff,PFOppRegDiff,FGPRegDiff,FGP3RegDiff,FTPRegDiff,FGPOppRegDiff,FGP3OppRegDiff,FTPOppRegDiff,ScoreTouDiff,ScoreOppTouDiff,NumOTTouDiff,FGMTouDiff,FGATouDiff,FGM3TouDiff,FGA3TouDiff,FTMTouDiff,FTATouDiff,ORTouDiff,DRTouDiff,AstTouDiff,TOTouDiff,StlTouDiff,BlkTouDiff,PFTouDiff,FGMOppTouDiff,FGAOppTouDiff,FGM3OppTouDiff,FGA3OppTouDiff,FTMOppTouDiff,FTAOppTouDiff,OROppTouDiff,DROppTouDiff,AstOppTouDiff,TOOppTouDiff,StlOppTouDiff,BlkOppTouDiff,PFOppTouDiff,FGPTouDiff,FGP3TouDiff,FTPTouDiff,FGPOppTouDiff,FGP3OppTouDiff,FTPOppTouDiff,SeedDiff,OrdinalRankDiff
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015_1326_1433,True,-3.0,4.0,0.0,-2.0,2.0,2.0,5.0,1.0,1.0,1.0,-1.0,-4.0,-1.0,1.0,-1.0,2.0,0.0,-2.0,-1.0,-2.0,3.0,6.0,0.0,5.0,0.0,2.0,0.0,1.0,1.0,-0.049955,0.014493,0.015152,0.015094,-0.017544,-0.064286,16.0,17.0,1.0,6.0,15.0,5.0,9.0,-1.0,-1.0,8.0,-7.0,1.0,-3.0,1.0,1.0,5.0,5.0,2.0,3.0,3.0,4.0,6.0,6.0,1.0,4.0,4.0,-4.0,2.0,2.0,-0.018462,0.130952,-0.030303,0.080432,0.144231,-0.025575,-3,4.0
2015_1301_1437,True,5.5,-3.5,0.0,0.5,-4.5,2.5,5.0,1.5,-1.5,-2.0,-0.5,4.0,0.0,3.0,-1.0,-2.0,0.0,0.5,1.0,0.0,-5.5,-6.0,-1.0,-2.5,0.5,3.5,1.0,-1.0,-1.0,0.045601,0.036232,0.128205,-0.003477,0.058824,-0.080952,-8.0,-6.0,-0.5,-5.5,0.0,3.5,15.5,-0.5,-6.5,1.0,1.0,-1.5,3.0,1.5,-1.0,3.5,-6.5,-2.0,2.0,6.0,5.0,0.5,0.0,0.5,0.0,0.5,2.5,0.0,-6.0,-0.097345,-0.070048,0.171226,-0.097260,-0.002841,0.219823,-7,-31.0
2010_1199_1211,False,10.0,10.0,0.0,3.0,-2.0,-1.0,-1.0,6.0,3.0,-1.0,2.0,-1.0,-3.0,-3.0,-3.0,0.0,4.0,5.0,0.0,2.0,0.0,1.0,-2.0,1.0,2.0,-2.0,-1.0,0.0,2.0,0.071429,-0.040441,0.178571,0.036409,-0.033333,-0.034211,18.0,20.0,-1.0,7.0,12.0,3.0,8.0,1.0,5.0,3.0,-2.0,9.0,-4.0,-1.0,1.0,2.0,10.0,8.0,2.0,-4.0,4.0,6.0,6.0,0.0,5.0,0.0,0.0,2.0,4.0,0.030735,0.025641,-0.137681,0.108722,0.144928,0.000000,-1,-1.0
2008_1140_1401,False,-3.5,-1.0,0.0,-1.0,-4.0,-2.0,-6.0,-1.5,-0.5,0.5,-2.0,-3.5,-0.5,-1.0,0.5,-2.0,0.5,0.0,1.0,1.0,-4.0,-3.0,0.0,-3.5,1.0,-3.0,-1.5,1.0,-1.5,0.015558,0.011364,-0.054264,0.008621,0.039216,-0.122222,-9.0,-14.0,0.0,-5.0,-16.0,-4.0,-9.0,5.0,14.0,-4.0,3.0,0.0,-2.0,2.0,-1.0,-3.0,-3.0,-2.0,-3.0,1.0,-9.0,-8.0,-5.0,-4.0,-4.0,0.0,-1.0,2.0,0.0,0.037752,0.026316,-0.213675,-0.039683,-0.192810,-0.126437,1,-4.0
2018_1300_1411,False,6.0,10.0,0.0,-0.5,2.0,2.0,4.0,3.5,3.0,-1.0,2.0,-2.0,-0.5,0.5,1.0,-2.0,5.0,5.0,1.0,-2.0,-1.0,1.5,3.0,2.0,2.5,0.5,0.0,1.0,3.0,-0.023937,0.039216,0.072967,0.046524,0.081818,-0.095349,1.0,36.0,0.0,-2.0,-7.0,2.0,1.0,3.0,3.0,-3.0,0.0,-5.0,4.0,-4.0,3.0,2.0,16.0,27.0,3.0,6.0,1.0,4.0,14.0,0.0,4.0,-8.0,2.0,3.0,0.0,0.008458,0.066952,0.043269,0.048333,0.051136,-0.087413,0,-49.5


In [13]:
y_train = features_train["Win"]
X_train = features_train.drop("Win", axis=1)
y_valid = features_valid["Win"]
X_valid = features_valid.drop("Win", axis=1)

# Step 4: Train a model


### Setup Hyperparameter Tuning
See https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning

In [14]:
from sklearn.metrics import brier_score_loss, roc_auc_score, confusion_matrix
from sklearn.model_selection import cross_val_score
import lightgbm

model = lightgbm.LGBMClassifier(objective="binary", min_data_in_leaf=200)
model.fit(X_train, y_train)

scores = cross_val_score(model, X_train, y_train, cv=5)
print(*scores)


[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
0.7090909090909091 0.7818181818181819 0.6909090909090909 0.6636363636363637 0.7818181818181819


In [15]:
outcomes = get_outcomes(data)
features_full = merge_outcomes_with_features(outcomes, features)
y = features_full["Win"]
X = features_full.drop("Win", axis=1)
model.fit(X, y)
preds = model.predict(X)
print("Training Score:", model.score(X, y))
print("Brier Score:", brier_score_loss(y, preds))
print("ROC AUC:", roc_auc_score(y, preds))
print("Parameters")
print(*(f"- {key}: {value}" for key, value in model.get_params(deep=True).items()), sep="\n")
print("Features")
print(*(f"- {name}: {imp}" for name, imp in sorted(zip(model.feature_name_, model.feature_importances_), key=lambda x: x[1], reverse=True)), sep="\n")


[LightGBM] [Warning] min_data_in_leaf is set=200, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=200
Training Score: 0.7858117326057299
Brier Score: 0.21418826739427013
ROC AUC: 0.7858148893360161
Parameters
- boosting_type: gbdt
- class_weight: None
- colsample_bytree: 1.0
- importance_type: split
- learning_rate: 0.1
- max_depth: -1
- min_child_samples: 20
- min_child_weight: 0.001
- min_split_gain: 0.0
- n_estimators: 100
- n_jobs: -1
- num_leaves: 31
- objective: binary
- random_state: None
- reg_alpha: 0.0
- reg_lambda: 0.0
- silent: warn
- subsample: 1.0
- subsample_for_bin: 200000
- subsample_freq: 0
- min_data_in_leaf: 200
Features
- SeedDiff: 22
- TOTouDiff: 13
- FGPRegDiff: 12
- FGMRegDiff: 9
- FGARegDiff: 8
- OROppTouDiff: 8
- TOOppTouDiff: 8
- PFRegDiff: 7
- FTATouDiff: 7
- ScoreOppRegDiff: 6
- FGMOppRegDiff: 6
- DROppRegDiff: 6
- ScoreTouDiff: 6
- FGM3RegDiff: 4
- ORRegDiff: 4
- OROppRegDiff: 4
- AstOppRegDiff: 4
- FTMTouDiff: 4
- AstTouDiff: 4
- FTA

# Step 5: Submit to the competition

We"ll begin by using the trained model to generate predictions, which we"ll save to a CSV file.

In [16]:
SampleSubmissionWarmup = pd.read_csv("/kaggle/input/warmup-round-march-machine-learning-mania-2023/SampleSubmissionWarmup.csv")

print(SampleSubmissionWarmup.shape)
SampleSubmissionWarmup.tail()

(614319, 2)


,ID,Pred
614314,2022_3469_3471,0.5
614315,2022_3469_3472,0.5
614316,2022_3470_3471,0.5
614317,2022_3470_3472,0.5
614318,2022_3471_3472,0.5


In [17]:




def get_submission_outcomes(sample_submission: pd.DataFrame) -> pd.DataFrame:
    df = sample_submission.copy()
    df.drop("Pred", axis=1, inplace=True)
    df[["Season", "LowID", "HighID"]] = df["ID"].str.split("_", expand=True)
    df[["Season", "LowID", "HighID"]] = df[["Season", "LowID", "HighID"]].astype(int)
    return df

In [18]:
submission_outcomes = get_submission_outcomes(SampleSubmissionWarmup)
print(submission_outcomes.shape)
submission_outcomes.tail()

(614319, 4)


,ID,Season,LowID,HighID
614314,2022_3469_3471,2022,3469,3471
614315,2022_3469_3472,2022,3469,3472
614316,2022_3470_3471,2022,3470,3471
614317,2022_3470_3472,2022,3470,3472
614318,2022_3471_3472,2022,3471,3472


In [19]:
X_submission = merge_outcomes_with_features(submission_outcomes, features, how="left").fillna(0)
print(X_submission.shape)
X_submission.tail()

(614319, 72)


,ScoreRegDiff,ScoreOppRegDiff,NumOTRegDiff,FGMRegDiff,FGARegDiff,FGM3RegDiff,FGA3RegDiff,FTMRegDiff,FTARegDiff,ORRegDiff,DRRegDiff,AstRegDiff,TORegDiff,StlRegDiff,BlkRegDiff,PFRegDiff,FGMOppRegDiff,FGAOppRegDiff,FGM3OppRegDiff,FGA3OppRegDiff,FTMOppRegDiff,FTAOppRegDiff,OROppRegDiff,DROppRegDiff,AstOppRegDiff,TOOppRegDiff,StlOppRegDiff,BlkOppRegDiff,PFOppRegDiff,FGPRegDiff,FGP3RegDiff,FTPRegDiff,FGPOppRegDiff,FGP3OppRegDiff,FTPOppRegDiff,ScoreTouDiff,ScoreOppTouDiff,NumOTTouDiff,FGMTouDiff,FGATouDiff,FGM3TouDiff,FGA3TouDiff,FTMTouDiff,FTATouDiff,ORTouDiff,DRTouDiff,AstTouDiff,TOTouDiff,StlTouDiff,BlkTouDiff,PFTouDiff,FGMOppTouDiff,FGAOppTouDiff,FGM3OppTouDiff,FGA3OppTouDiff,FTMOppTouDiff,FTAOppTouDiff,OROppTouDiff,DROppTouDiff,AstOppTouDiff,TOOppTouDiff,StlOppTouDiff,BlkOppTouDiff,PFOppTouDiff,FGPTouDiff,FGP3TouDiff,FTPTouDiff,FGPOppTouDiff,FGP3OppTouDiff,FTPOppTouDiff,SeedDiff,OrdinalRankDiff
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022_3469_3471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022_3469_3472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022_3470_3471,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022_3470_3472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022_3471_3472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# Use the model to generate predictions

predictions = model.predict_proba(X_submission)[:,1]

# Save the predictions to a CSV file
output = pd.DataFrame({"ID": X_submission.index,
                       "Pred": predictions})
output.to_csv("submission.csv", index=False)
print(output.shape)
output.describe()

(614319, 2)


,Pred
count,614319.000000
mean,0.688762
std,0.028878
min,0.048936
25%,0.690212
50%,0.690212
75%,0.690212
max,0.949400
